In [2]:
!pip install matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 1.8 MB/s eta 0:00:00a 0:00:01
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 2.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.0/245.0 kB 2.5 MB/s eta 0:00:00a 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 2.3 MB/s eta 0:00:00
Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)


In [4]:
!pip install llama_index

  Using cached llama_index-0.10.55-py3-none-any.whl.metadata (11 kB)
  Using cached llama_index_agent_openai-0.2.8-py3-none-any.whl.metadata (729 bytes)
  Using cached llama_index_cli-0.1.12-py3-none-any.whl.metadata (1.5 kB)
  Using cached llama_index_core-0.10.55-py3-none-any.whl.metadata (2.4 kB)
  Using cached llama_index_embeddings_openai-0.1.10-py3-none-any.whl.metadata (604 bytes)
  Using cached llama_index_indices_managed_llama_cloud-0.2.5-py3-none-any.whl.metadata (3.8 kB)
  Using cached llama_index_legacy-0.9.48-py3-none-any.whl.metadata (8.5 kB)
  Using cached llama_index_llms_openai-0.1.26-py3-none-any.whl.metadata (610 bytes)
  Using cached llama_index_multi_modal_llms_openai-0.1.8-py3-none-any.whl.metadata (728 bytes)
  Using cached llama_index_program_openai-0.1.6-py3-none-any.whl.metadata (715 bytes)
  Using cached llama_index_question_gen_openai-0.1.3-py3-none-any.whl.metadata (785 bytes)
  Using cached llama_index_readers_file-0.1.30-py3-none-any.whl.metadata (5.4 kB)

In [2]:
import os

import numpy as np

from llama_index.core import Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.node_parser import HierarchicalNodeParser
from llama_index.core.node_parser import get_leaf_nodes
from llama_index.core import StorageContext
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core import  load_index_from_storage
from llama_index.core import ServiceContext, VectorStoreIndex, StorageContext
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core import load_index_from_storage
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.schema import NodeWithScore
from copy import deepcopy
from llama_index.core.indices.postprocessor import SentenceTransformerRerank
from llama_index.core import QueryBundle
from llama_index.core.schema import TextNode, NodeWithScore
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor
from llama_index.core.response.notebook_utils import display_response

In [3]:
def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context, service_context=merging_context
        )
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index


def get_automerging_query_engine(
    automerging_index,
    similarity_top_k=12,
    rerank_top_n=6,
):
    base_retriever = automerging_index.as_retriever(similarity_top_k=similarity_top_k)
    retriever = AutoMergingRetriever(
        base_retriever, automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine = RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [3]:
documents = SimpleDirectoryReader(
    input_files=["/Users/rohit/Desktop/Juspay/data/Ec Headless - Android _ Juspay Developer Docs.pdf"]
).load_data()

In [4]:
document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [9]:
!pip install langchain_community

  Using cached langchain_community-0.2.7-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain-0.2.9-py3-none-any.whl.metadata (6.9 kB)
  Using cached langchain_core-0.2.21-py3-none-any.whl.metadata (6.0 kB)
  Using cached langsmith-0.1.92-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl.metadata (2.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 883.4 kB/s eta 0:00:00a 0:00:01
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
Using cached langchain_community-0.2.7-py3-none-any.whl (2.2 MB)
Using cached langchain-0.2.9-py3-none-any.whl (987 kB)
Using cached langchain_core-0.2.21-py3-none-any.whl (372 kB)
Using cached langsmith-0.1.92-py3-none-any.whl (134 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached langchain_text_splitters-0.2.2-py3-none-any.whl (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [7]:
from langchain_community.chat_models import ChatOllama
local_llm = 'llama3'
llm = ChatOllama(model=local_llm, temperature=0)

In [12]:
!pip install llama-index-llms-langchain

  Using cached llama_index_llms_langchain-0.1.4-py3-none-any.whl.metadata (751 bytes)
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached llama_index_llms_anyscale-0.1.4-py3-none-any.whl.metadata (647 bytes)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl.metadata (2.2 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
Using cached llama_index_llms_langchain-0.1.4-py3-none-any.whl (4.8 kB)
Using cached langchain-0.1.20-py3-none-any.whl (1.0 MB)
Using cached llama_index_llms_anyscale-0.1.4-py3-none-any.whl (4.2 kB)
Using cached langchain_community-0.0.38-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
Using cached langchain_text_splitters-0.0.2-py3-none-any.whl (23 kB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninsta

In [13]:
!pip install llama-index-embeddings-huggingface

  Using cached llama_index_embeddings_huggingface-0.2.2-py3-none-any.whl.metadata (769 bytes)
  Using cached huggingface_hub-0.24.0-py3-none-any.whl.metadata (13 kB)
  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached filelock-3.15.4-py3-none-any.whl.metadata (2.9 kB)
  Using cached minijinja-2.0.1-cp38-abi3-macosx_10_12_x86_64.macosx_11_0_arm64.macosx_10_12_universal2.whl.metadata (8.8 kB)
  Using cached transformers-4.42.4-py3-none-any.whl.metadata (43 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Using cached sympy-1.13.0-py3-none-any.whl.metadata (12 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached llama_index_embeddings_huggingface-0.2.2-py3-none-any.whl (7.2 kB)
Using cached huggingface_hub-0.24.0-py3-none-any.whl (419 kB)
Using cac

In [8]:
sentence_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    node_parser=node_parser,
)

/var/folders/t_/zhkb2m0s0494lhtdscp9vm080000gn/T/ipykernel_3995/760635039.py:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(
/opt/miniconda3/envs/juspay39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
!pip install llama-index-llms-fireworks

In [9]:
sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [10]:
sentence_index.storage_context.persist(persist_dir="./sentence_index")

In [11]:
if not os.path.exists("./sentence_index"):
    sentence_index = VectorStoreIndex.from_documents(
        [document], service_context=sentence_context
    )

    sentence_index.storage_context.persist(persist_dir="./sentence_index")
else:
    sentence_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./sentence_index"),
        service_context=sentence_context
    )

In [12]:
postproc = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)

In [13]:
rerank = SentenceTransformerRerank(
    top_n=2, model="BAAI/bge-reranker-base"
)

In [14]:
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k=6, node_postprocessors=[postproc, rerank]
)

In [17]:
window_response = sentence_window_engine.query(
    "What is the limit for number of transactions?"
)

In [18]:
display_response(window_response)

**`Final Response:`** The number of transactions are capped at 200 per day.